Criar tabela top 10 (lista com 10 posições)
Ranking de similaridade

Bow binário X tf-idf

Vou ter uma query contendo um texto e uma categoria. Depois de montar ranking de similaridade, se for da mesma categoria, incrementa no vetor top10.

Aluna: Carolina Estrella

**Matrícula: 180074792**

In [24]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk.corpus import reuters
import math
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn

In [25]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

##Pre-Processing

In [48]:
def pre_processing(text):
  stopwords_nltk = stopwords.words('english')
  stemmer = PorterStemmer()
  wn_lemmas = set(wn.all_lemma_names())
  
  words = re.sub(r'[\s!%*~\^´`=+<\[\]?&$:;@#.0-9()\/\"\'_-]+|[\s]*[\\]u[a-z0-9]{4}[\s]*|[\s]*http[^\s]+[\s]*', " ", text.lower())
  words = re.sub(r'\brt\b', "", words)
  words = re.sub(r'[\\]+', " ", words)
  words = words.strip()
  
  phrase = ""
  words = words.split()

  for x in words:
    if x not in stopwords_nltk and x in wn_lemmas:
      x = stemmer.stem(x)
      phrase = phrase + " " + x
        
  return phrase.strip()

In [50]:
categories = []
corpus = []
fileids = reuters.fileids()

for file in fileids:
  categories.append(reuters.categories(file))
  corpus.append(pre_processing(reuters.raw(file)))

df = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':corpus})

In [51]:
df

,ids,categories,text
0,test/14826,[trade],asian fear damag u japan rift mount trade fric...
1,test/14828,[grain],china daili vermin eat pct grain stock survey ...
2,test/14829,"[crude, nat-gas]",japan revis long term energi demand downward m...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",thai trade deficit first quarter thailand trad...
4,test/14833,"[palm-oil, veg-oil]",indonesia price rise sharpli indonesia crude p...
...,...,...,...
10783,training/999,"[interest, money-fx]",u k money market shortag forecast revis bank e...
10784,training/9992,[earn],knight inc quarterli prior pay april record ap...
10785,training/9993,[earn],inc quarterli prior pay april record april seven
10786,training/9994,[earn],nationwid cellular servic inc th loss six loss...


In [52]:
phrase = " ".join(corpus)
tokens = phrase.split()
vocab = sorted(set(tokens))

##Bag of Words Binário

In [30]:
bow_binario = {}

for i,doc in enumerate(corpus):
  bow_binario[i] = dict()
  for word in doc.split():
    bow_binario[i][word] = 1

In [ ]:
bow_binario

##Bag of Words Ponderado

In [31]:
bow_ponderado = {}
N = {} # quantidade de termos em cada documento

for i,doc in enumerate(corpus):
    bow_ponderado[i] = dict()
    N[i] = len(doc.split())
    for word in doc.split():
        if word in bow_ponderado[i]:
            bow_ponderado[i][word] += 1
        else:
            bow_ponderado[i][word] = 1

for doc in bow_ponderado:
    for word in bow_ponderado[doc]:
        bow_ponderado[doc][word] /= N[doc]

In [ ]:
bow_ponderado

##Bag of Words Contagem

In [32]:
bow_contagem = {}

for i,doc in enumerate(corpus):
    bow_contagem[i] = dict()
    for word in doc.split():
        if word in bow_contagem[i]:
            bow_contagem[i][word] += 1
        else:
            bow_contagem[i][word] = 1

In [ ]:
bow_contagem

##Inverse Document Frequency

In [34]:
idf = {}
for word in vocab:
  idf[word] = 0
  #contabilizar em quantos documentos a palavra word aparece
  for doc in bow_contagem:
    if word in bow_contagem[doc]:
      idf[word] += 1

for word in idf:
  if(idf[word] == 0): continue
  idf[word] = math.log(len(corpus)/idf[word])

In [ ]:
idf

##Bag of Words TFIDF

In [36]:
bow_tfidf = {}

for doc in bow_ponderado:
  bow_tfidf[doc] = dict()
  for word in bow_ponderado[doc]:
    bow_tfidf[doc][word] = bow_ponderado[doc][word]*idf[word]

In [37]:
tfidf = pd.DataFrame().from_records(bow_tfidf).fillna(0).T

In [ ]:
tfidf

##Cosine Similarity

In [39]:
def cosine_dissimilarity(vector1, vector2):
  prod_interno = 0

  for i, value in enumerate(vector1):
    prod_interno += value*vector2[i]

  norma_vector1 = 0

  for x in vector1:
    norma_vector1 += x*x
  
  norma_vector1 = norma_vector1**0.5

  #norma_vector1 = math.sqrt(sum([x**2 for x in vector1]))

  norma_vector2 = 0

  for x in vector1:
    norma_vector2 += x*x
  
  norma_vector2 = norma_vector2**0.5

  return prod_interno/(norma_vector1*norma_vector2)

##Recuperação de textos

In [67]:
top10 = [0] * 10

category = categories[:100]

for i in range(len(category)):
  vector1 = tfidf.iloc[i]
  results = []

  for j in range(len(category)):
    if(i == j): continue
    vector2 = tfidf.iloc[j]
    similarity = cosine_dissimilarity(vector1, vector2)
    results.append((similarity,j))

  results.sort(reverse=True)
  best10 = results[:10]
  for x, [sim,j] in enumerate(best10):
    for c in category[j]:
      if(c in category[i]): top10[x] += 1


print(top10)

[78, 73, 60, 54, 53, 45, 49, 49, 45, 35]
